In [51]:
import numpy as np 
import pandas as pd
import statsmodels.api as sm

<h1>Load in the Auto MPG dataset</h1>

In [1]:
!pip install ucimlrepo


[notice] A new release of pip available: 23.1.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [34]:
# Import the dataset
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
auto_mpg = fetch_ucirepo(id=9) 
  
# data (as pandas dataframes) 
X = auto_mpg.data.features 
y = auto_mpg.data.targets 
  
# variable information 
print(auto_mpg.variables) 


           name     role         type demographic description units  \
0  displacement  Feature   Continuous        None        None  None   
1           mpg   Target   Continuous        None        None  None   
2     cylinders  Feature      Integer        None        None  None   
3    horsepower  Feature   Continuous        None        None  None   
4        weight  Feature   Continuous        None        None  None   
5  acceleration  Feature   Continuous        None        None  None   
6    model_year  Feature      Integer        None        None  None   
7        origin  Feature      Integer        None        None  None   
8      car_name       ID  Categorical        None        None  None   

  missing_values  
0             no  
1             no  
2             no  
3            yes  
4             no  
5             no  
6             no  
7             no  
8             no  


<h1>Brief Feature Engineering</h1>

In [35]:
# Change the values of the origin column to represent the country of origin as strings: 'US', 'Europe', and 'Asia'
X['origin'] = X['origin'].replace({1: 'US', 2: 'Europe', 3: 'Asia'})

# One-hot encode the origin column
X = pd.get_dummies(X, columns=['origin'], drop_first=True)

/Users/dylanskinner/opt/anaconda3/envs/acme1/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [36]:
# See how many NaN's are present in the horsepower column.
print(X.horsepower.isna().sum())

# Get the indices of the rows containing NaN's
nan_indices = X[X.horsepower.isna()].index

# Since there are only 6 NaN's, we can drop them from the dataset (also dropping the corresponding rows from y)
X = X.dropna()
y = y.drop(nan_indices)

6


In [37]:
# Add a constant column to the dataframe.
X = sm.add_constant(X)

<h1>Stepwise Feature Removal</h1>

In [38]:
# Fit the model
model = sm.OLS(y, X).fit()

In [40]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.824
Model:                            OLS   Adj. R-squared:                  0.821
Method:                 Least Squares   F-statistic:                     224.5
Date:                Wed, 03 Jan 2024   Prob (F-statistic):          1.79e-139
Time:                        09:43:04   Log-Likelihood:                -1020.5
No. Observations:                 392   AIC:                             2059.
Df Residuals:                     383   BIC:                             2095.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const           -15.1014      4.681     -3.226

In [47]:
# Drop the 'origin_Europe' column
X1 = X.drop('origin_Europe', axis=1)

# Refit the model and print the summary
model1 = sm.OLS(y, X1).fit()
print(model1.summary())


                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.824
Model:                            OLS   Adj. R-squared:                  0.821
Method:                 Least Squares   F-statistic:                     257.1
Date:                Wed, 03 Jan 2024   Prob (F-statistic):          1.16e-140
Time:                        10:00:11   Log-Likelihood:                -1020.5
No. Observations:                 392   AIC:                             2057.
Df Residuals:                     384   BIC:                             2089.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const          -15.4664      4.583     -3.375  

In [48]:
# Drop the 'acceleration' column
X2 = X1.drop('acceleration', axis=1)

# Refit the model and print the summary
model2 = sm.OLS(y, X2).fit()
print(model2.summary())


                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.824
Model:                            OLS   Adj. R-squared:                  0.821
Method:                 Least Squares   F-statistic:                     300.1
Date:                Wed, 03 Jan 2024   Prob (F-statistic):          8.89e-142
Time:                        10:03:36   Log-Likelihood:                -1020.9
No. Observations:                 392   AIC:                             2056.
Df Residuals:                     385   BIC:                             2084.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const          -13.8400      4.113     -3.365  

In [53]:
# Drop the 'cylinders' column
X3 = X2.drop('cylinders', axis=1)

# Refit the model and print the summary
model3 = sm.OLS(y, X3).fit()
print()
print(model3.summary())


                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.823
Model:                            OLS   Adj. R-squared:                  0.820
Method:                 Least Squares   F-statistic:                     358.3
Date:                Wed, 03 Jan 2024   Prob (F-statistic):          1.48e-142
Time:                        10:11:05   Log-Likelihood:                -1022.1
No. Observations:                 392   AIC:                             2056.
Df Residuals:                     386   BIC:                             2080.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const          -15.0080      4.051     -3.705  

In [65]:
# Perform train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=334)
y_test = y_test.values

# Train the first model, and get the MSE on the test set
model = sm.OLS(y_train, X_train).fit()
y_pred = model.predict(X_test).values
mse = np.mean((y_pred - y_test)**2)
print(f'MSE when using all features:         {mse}')

# Now, remove the features we decided weren't useful through stepwise feature removal, and get the MSE on the test set
X_train = X_train.drop(['origin_Europe', 'acceleration', 'cylinders'], axis=1)
X_test = X_test.drop(['origin_Europe', 'acceleration', 'cylinders'], axis=1)
model = sm.OLS(y_train, X_train).fit()
y_pred = model.predict(X_test).values
mse = np.mean((y_pred - y_test)**2)
print(f'MSE when using only useful features: {mse}')

MSE when using all features:         99.67098477764772
MSE when using only useful features: 98.94744747911336


# $L^1$ Regularization

In [66]:
from sklearn.linear_model import LassoLarsIC

In [109]:
lars_model = LassoLarsIC(criterion='bic', normalize=False)

In [127]:
X = X.drop('const', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=334)
y_test = y_test.values

In [128]:
# Fit the model
lars_model.fit(X_train, y_train.values.reshape(1, -1)[0])

LassoLarsIC(criterion='bic', normalize=False)

In [129]:
# Get the results from the model
results = pd.DataFrame(
    {
        "alphas": lars_model.alphas_,
        "BIC criterion": lars_model.criterion_,
    }
).set_index("alphas")
print(results)

             BIC criterion
alphas                    
5791.332580    3181.170871
23.835366      1837.591358
15.655978      1838.232508
8.145562       1831.269152
2.384037       1674.376395
0.663504       1660.903465
0.295187       1665.666492
0.232070       1662.522325
0.188430       1663.228888
0.033225       1655.087183
0.000000       1659.478067


In [131]:
# Get coefficients from the model
lars_model.coef_


array([ 0.02359115, -0.61367486, -0.01471611, -0.00695722,  0.04982891,
        0.74492259,  0.        , -2.30763917])

In [136]:
from sklearn.linear_model import LinearRegression
lasso_no_alpha = LinearRegression().fit(X_train, y_train.values.reshape(1, -1)[0])

# Get the MSE on the test set for each model
y_pred_no_alpha = lasso_no_alpha.predict(X_test)
y_pred_opt_alpha = lars_model.predict(X_test)

mse_no_alpha = np.mean((y_pred_no_alpha - y_test)**2)
mse_opt_alpha = np.mean((y_pred_opt_alpha - y_test)**2)

print(f'MSE when alpha=0:         {mse_no_alpha}')
print(f'MSE when alpha=0.033225:  {mse_opt_alpha}')

MSE when alpha=0:         99.6709847776471
MSE when alpha=0.033225:  98.75552193526808
